In [4]:
import os

spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [5]:
# Start Spark session
from pyspark.sql import SparkSession
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-04 02:18:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.67MB/s    in 0.5s    

2022-11-04 02:18:50 (1.67 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract


In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/county_pop_df.csv"
spark.sparkContext.addFile(url)
county_pop_df = spark.read.csv(SparkFiles.get("county_pop_df.csv"), sep=",", header=True,)

In [8]:
county_pop_df.show(5)

+---------+----------+----------+---------------+-------------------+-----------------+---------------+-------------+----------------+--------------+---------------+-------------+----------------+--------------+-----------------+---------------+------------------+----------------+-------------+-----------+-----------------+---------------+--------------------+------------------+----------------------+--------------------+--------------+------------+---------------+-------------+---------------+-------------+------------------+----------------+----------------+--------------+--------------------+------------------+------------------+----------------+---------------+-------------+------------------+----------------+----------------+--------------+-------------+-----------+------------------+----------------+---------------+-------------+-----------------+---------------+---------------+-------------+---------------+-------------+----------------+--------------+-------------------+-------

# Transform


In [9]:
county_pop_df = county_pop_df.dropna()

In [10]:
county_pop_df.printSchema()

root
 |-- StateDesc: string (nullable = true)
 |-- CountyName: string (nullable = true)
 |-- CountyFIPS: string (nullable = true)
 |-- TotalPopulation: string (nullable = true)
 |-- ARTHRITIS_CrudePrev: string (nullable = true)
 |-- ARTHRITIS_AdjPrev: string (nullable = true)
 |-- BINGE_CrudePrev: string (nullable = true)
 |-- BINGE_AdjPrev: string (nullable = true)
 |-- BPHIGH_CrudePrev: string (nullable = true)
 |-- BPHIGH_AdjPrev: string (nullable = true)
 |-- BPMED_CrudePrev: string (nullable = true)
 |-- BPMED_AdjPrev: string (nullable = true)
 |-- CANCER_CrudePrev: string (nullable = true)
 |-- CANCER_AdjPrev: string (nullable = true)
 |-- CASTHMA_CrudePrev: string (nullable = true)
 |-- CASTHMA_AdjPrev: string (nullable = true)
 |-- CERVICAL_CrudePrev: string (nullable = true)
 |-- CERVICAL_AdjPrev: string (nullable = true)
 |-- CHD_CrudePrev: string (nullable = true)
 |-- CHD_AdjPrev: string (nullable = true)
 |-- CHECKUP_CrudePrev: string (nullable = true)
 |-- CHECKUP_AdjPrev

# Load

In [11]:
from getpass import getpass

password = getpass('enter pass words here')

enter pass words here··········


In [12]:
#***Load
#configure settings for RDS
mode="append"
jdbc_url = "jdbc:postgresql://b-team-final-project.cct7ahzel1ur.us-west-2.rds.amazonaws.com:5432/B_Team_Final_Project"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [16]:
#load DataFrames into pgAdmin
# Write DataFrame to customers_table in RDS
county_pop_df.write.jdbc(url=jdbc_url, table='places_county', mode=mode, properties=config)